In [ ]:
import os
import sys
from pathlib import Path
import geopandas as gpd

parent_dir = Path().resolve().parent
sys.path.insert(0, str(parent_dir))
from map2grid import *

In [11]:
data_ways_all_gdf = gpd.read_file("../data/osm/vietnam-latest_lines.gpkg")
data_ways_all_gdf = data_ways_all_gdf.to_crs(epsg=32648)
print(data_ways_all_gdf.crs)

EPSG:32648


In [ ]:
# nodes_gdf = extract_unique_points(data_ways_all_gdf)
# print(nodes_gdf.head())

In [4]:
nodes_gdf = extract_unique_endpoints(data_ways_all_gdf)  # 提取唯一的起点和终点坐标
print(nodes_gdf.head())

   nodeID            lon           lat                        geometry
0       0  610714.852695  2.315181e+06  POINT (610714.853 2315180.700)
1       1  609979.512105  2.317122e+06  POINT (609979.512 2317121.769)
2       2  672622.408581  1.278800e+06  POINT (672622.409 1278800.027)
3       3  651889.179899  1.285444e+06  POINT (651889.180 1285443.951)
4       4  682154.017051  2.315888e+06  POINT (682154.017 2315887.790)


In [5]:
updated_gdf = add_node_ids_to_lines(data_ways_all_gdf, nodes_gdf)  # 为原始数据添加node1和node2列
updated_gdf.head()

/scistor/ivm/mye500/miniconda3/envs/py310/lib/python3.10/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
/scistor/ivm/mye500/miniconda3/envs/py310/lib/python3.10/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


,osm_id,power,voltage,utility,name,frequency,plant_output_electricity,plant_source,plant_method,generator_output_electricity,...,generator_type,substation,location,cables,circuits,line,layer,geometry,node1,node2
0,153608701,line,110000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6,2,NaN,NaN,"LINESTRING (610714.853 2315180.700, 610719.861...",0,1
1,176708144,line,110000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (672622.409 1278800.027, 672372.817...",2,3
2,241194661,line,220000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6,2,NaN,NaN,"LINESTRING (682154.017 2315887.790, 682153.010...",4,5
3,241255651,line,110000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (684889.726 2304273.827, 684912.615...",6,7
4,241255652,line,220000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6,NaN,NaN,NaN,"LINESTRING (684710.914 2304282.784, 684525.431...",8,9


In [6]:
updated_gdf['Length'] = updated_gdf['geometry'].length * 1.2 / 1000

In [7]:
updated_gdf.head()

,osm_id,power,voltage,utility,name,frequency,plant_output_electricity,plant_source,plant_method,generator_output_electricity,...,substation,location,cables,circuits,line,layer,geometry,node1,node2,Length
0,153608701,line,110000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,NaN,NaN,6,2,NaN,NaN,"LINESTRING (610714.853 2315180.700, 610719.861...",0,1,2.692679
1,176708144,line,110000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (672622.409 1278800.027, 672372.817...",2,3,27.367648
2,241194661,line,220000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,NaN,NaN,6,2,NaN,NaN,"LINESTRING (682154.017 2315887.790, 682153.010...",4,5,16.135774
3,241255651,line,110000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (684889.726 2304273.827, 684912.615...",6,7,2.613165
4,241255652,line,220000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,NaN,NaN,6,NaN,NaN,NaN,"LINESTRING (684710.914 2304282.784, 684525.431...",8,9,22.028022


In [8]:
data_ways_all_voltage = count_voltage_levels(data_ways_all_gdf)

Start counting voltage levels...
Voltage levels count:
Voltage level 110 kV: 2073
Voltage level 220 kV: 980
Voltage level 500 kV: 2986
Count of rows with None voltage: 1782


In [9]:
data_ways_busbar, data_busbars = delete_busbars(data_ways_all_voltage)

Start deleting ways with type "busbar" or "bay"...
   ATTENTION! Way Element ID 1196308448 has type "busbar" or "bay", but is too long.
               Length: 0.51 km of max. 0.5 km
               This way won't be added to the "busbar" exception list.
   ATTENTION! Way Element ID 1271355297 has type "busbar" or "bay", but is too long.
               Length: 32.37 km of max. 0.5 km
               This way won't be added to the "busbar" exception list.
   ATTENTION! Way Element ID 1293378775 has type "busbar" or "bay", but is too long.
               Length: 31.64 km of max. 0.5 km
               This way won't be added to the "busbar" exception list.
   ATTENTION! Way Element ID 1309081709 has type "busbar" or "bay", but is too long.
               Length: 0.55 km of max. 0.5 km
               This way won't be added to the "busbar" exception list.
   ... there are 1354 busbars/bays in total
   ... 1350 busbars have been deleted
   ... finished! (0.345 seconds) 



In [10]:
data_ways_dc, dc_candidates = count_possible_dc(data_ways_busbar)

Start detecting lines which could be DC lines...
   ... 658 ways could potentially be a DC line.
   ... Please refer to dc_candidates for further checks.
   ... finished! (0.340 seconds) 



In [11]:
data_ways_cables = count_cables(data_ways_dc)

Start counting cables per way...
   ATTENTION! Unknown cable number ("nan") in ID 176708144. This way wont be cloned automatically.
   ATTENTION! Unknown cable number ("nan") in ID 241255651. This way wont be cloned automatically.
   ATTENTION! Unknown cable number ("nan") in ID 306551278. This way wont be cloned automatically.
   ATTENTION! Unknown cable number ("nan") in ID 306551280. This way wont be cloned automatically.
   ATTENTION! Unknown cable number ("nan") in ID 307638313. This way wont be cloned automatically.
   ATTENTION! Unknown cable number ("nan") in ID 307638318. This way wont be cloned automatically.
   ATTENTION! Unknown cable number ("nan") in ID 307777845. This way wont be cloned automatically.
   ATTENTION! Unknown cable number ("nan") in ID 307777860. This way wont be cloned automatically.
   ATTENTION! Unknown cable number ("nan") in ID 400152692. This way wont be cloned automatically.
   ATTENTION! Unknown cable number ("nan") in ID 400539088. This way wont be

In [12]:
M = my_calc_distances_between_endpoints(data_ways_cables)

In [13]:
data_ways_stacked, nodes_stacked_pairs = my_calc_stacked_endnodes(data_ways_cables, M)

Start finding all stacked endnodes...
... 9428 endnodes are stacked!


/scistor/ivm/mye500/projects/cascading_failure/power_flow_model/conversion_gridtool.py:816: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.at[i_ways, 'node1_stacked'] = False
/scistor/ivm/mye500/projects/cascading_failure/power_flow_model/conversion_gridtool.py:817: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.at[i_ways, 'node2_stacked'] = True


... finished! (0.626 seconds)



In [14]:
# 暂时省略my_calc_neighbouring_endnodes()

In [15]:
nodes_stacked_grouped = my_group_nodes(nodes_stacked_pairs)

Start grouping all pairs from "pairs_input" (may take a few seconds)...
   ... 9428 nodes will be grouped together in 3077 groups,
       with an average of 3.06 nodes per group.
   ... finished! (2.052 seconds)



In [16]:
data_ways_group_stacked_nodes = group_stacked_endnodes(data_ways_cables, nodes_gdf, nodes_stacked_grouped)

Completed updating coordinates for stacked groups.


In [17]:
data_ways_final_coords = my_add_final_coordinates(data_ways_group_stacked_nodes)

Start adding final coordinates...
... finished!


In [18]:
data_ways_delete, data_singular_ways = my_delete_singular_ways(data_ways_final_coords, 'ID_node1_final', 'ID_node2_final')

Start deleting ways which have the same endpoints after grouping...
   ... 0 ways were deleted!
   ... finished! (0.008 seconds)


In [19]:
data_ways_delete

,osm_id,power,voltage,utility,name,frequency,plant_output_electricity,plant_source,plant_method,generator_output_electricity,...,lon1_grouped,lat1_grouped,lon2_grouped,lat2_grouped,ID_node1_final,lon1_final,lat1_final,ID_node2_final,lon2_final,lat2_final
0,153608701,line,110000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,NaN,NaN,609979.512105,2.317122e+06,0.0,610714.852695,2.315181e+06,1.0,609979.512105,2.317122e+06
1,176708144,line,110000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,672622.408581,1.278800e+06,3.0,651889.179899,1.285444e+06
2,241194661,line,220000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,682154.017051,2.315888e+06,684741.069151,2.304331e+06,4.0,682154.017051,2.315888e+06,5.0,684741.069151,2.304331e+06
3,241255651,line,110000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,NaN,NaN,683277.940869,2.305527e+06,6.0,684889.726446,2.304274e+06,7.0,683277.940869,2.305527e+06
4,241255652,line,220000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,684710.914392,2.304283e+06,672301.251944,2.304724e+06,8.0,684710.914392,2.304283e+06,9.0,672301.251944,2.304724e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6466,1320784122,line,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,8648.0,618321.561237,1.090446e+06,8649.0,641674.201140,1.095248e+06
6467,1320784124,line,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,8650.0,658012.367671,1.079258e+06,8651.0,654930.192699,1.078817e+06
6468,1321731625,line,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,8652.0,753824.341205,1.719966e+06,8653.0,711286.966889,1.722638e+06
6469,1321731650,line,220000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,NaN,NaN,780561.152853,1.629737e+06,8654.0,731196.555863,1.654755e+06,6593.0,780561.152853,1.629737e+06


In [21]:
osm_subs = load_and_transform_osm_subs('power_data/vietnam-latest_substation_supplyID.gpkg')


EPSG:32648


/scistor/ivm/mye500/miniconda3/envs/py310/lib/python3.10/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


In [22]:
osm_subs.to_file('../outputs/osm_subs.gpkg', layer='osm_subs', driver='GPKG',espsg='32648')

In [24]:
data_ways_osm_subs, nodes_in_osm_subs = add_osm_ids_to_data(data_ways_delete, osm_subs)

In [25]:
data_ways_osm_subs

,osm_id,power,voltage,utility,name,frequency,plant_output_electricity,plant_source,plant_method,generator_output_electricity,...,lat1_final,ID_node2_final,lon2_final,lat2_final,osmID_node1,osmID_node2,lon1_final_1,lat1_final_1,lon2_final_1,lat2_final_1
0,153608701,line,110000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,2.315181e+06,1.0,609979.512105,2.317122e+06,153608703,1.0,610733.414911,2315123.836478,609979.512105,2317121.768718
1,176708144,line,110000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,1.278800e+06,3.0,651889.179899,1.285444e+06,2.0,1197858380,672622.408581,1278800.027259,651900.198685,1285465.579899
2,241194661,line,220000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,2.315888e+06,5.0,684741.069151,2.304331e+06,4.0,241065371,682154.017051,2315887.789708,684823.251473,2304280.331173
3,241255651,line,110000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,2.304274e+06,7.0,683277.940869,2.305527e+06,241065371,7.0,684823.251473,2304280.331173,683277.940869,2305527.150379
4,241255652,line,220000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,2.304283e+06,9.0,672301.251944,2.304724e+06,241065371,549914200,684823.251473,2304280.331173,672360.737403,2304845.086324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6466,1320784122,line,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.090446e+06,8649.0,641674.201140,1.095248e+06,1320784118,1052065872,618280.787307,1090388.227088,641629.570071,1095316.038624
6467,1320784124,line,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.079258e+06,8651.0,654930.192699,1.078817e+06,1318266947,8651.0,658018.11504,1079230.702641,654930.192699,1078816.969559
6468,1321731625,line,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.719966e+06,8653.0,711286.966889,1.722638e+06,8652.0,8653.0,753824.341205,1719965.524842,711286.966889,1722637.912955
6469,1321731650,line,220000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,1.654755e+06,6593.0,780561.152853,1.629737e+06,8654.0,6593.0,731196.555863,1654754.668432,780561.152853,1629736.774276


In [26]:
data_ways_delete2, data_singular_ways2 = my_delete_singular_ways(data_ways_osm_subs, 'osmID_node1', 'osmID_node2')

Start deleting ways which have the same endpoints after grouping...
   ... 780 ways were deleted!
   ... finished! (0.006 seconds)


In [27]:
data_ways_delete2

,osm_id,power,voltage,utility,name,frequency,plant_output_electricity,plant_source,plant_method,generator_output_electricity,...,lat1_final,ID_node2_final,lon2_final,lat2_final,osmID_node1,osmID_node2,lon1_final_1,lat1_final_1,lon2_final_1,lat2_final_1
0,153608701,line,110000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,2.315181e+06,1.0,609979.512105,2.317122e+06,153608703,1.0,610733.414911,2315123.836478,609979.512105,2317121.768718
1,176708144,line,110000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,1.278800e+06,3.0,651889.179899,1.285444e+06,2.0,1197858380,672622.408581,1278800.027259,651900.198685,1285465.579899
2,241194661,line,220000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,2.315888e+06,5.0,684741.069151,2.304331e+06,4.0,241065371,682154.017051,2315887.789708,684823.251473,2304280.331173
3,241255651,line,110000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,2.304274e+06,7.0,683277.940869,2.305527e+06,241065371,7.0,684823.251473,2304280.331173,683277.940869,2305527.150379
4,241255652,line,220000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,2.304283e+06,9.0,672301.251944,2.304724e+06,241065371,549914200,684823.251473,2304280.331173,672360.737403,2304845.086324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5686,1320784122,line,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.090446e+06,8649.0,641674.201140,1.095248e+06,1320784118,1052065872,618280.787307,1090388.227088,641629.570071,1095316.038624
5687,1320784124,line,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.079258e+06,8651.0,654930.192699,1.078817e+06,1318266947,8651.0,658018.11504,1079230.702641,654930.192699,1078816.969559
5688,1321731625,line,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.719966e+06,8653.0,711286.966889,1.722638e+06,8652.0,8653.0,753824.341205,1719965.524842,711286.966889,1722637.912955
5689,1321731650,line,220000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,1.654755e+06,6593.0,780561.152853,1.629737e+06,8654.0,6593.0,731196.555863,1654754.668432,780561.152853,1629736.774276


In [28]:
data_cloned_way = add_lineID_clone_ways(data_ways_delete2, 'VN')

Start adding "LineID" and cloning ways...
   ... 1766 ways doubled, 39 tripled, 118 quadrupled.
   ... finished! (2.998 seconds) 



In [29]:
data_cloned_way

,osm_id,power,voltage,utility,name,frequency,plant_output_electricity,plant_source,plant_method,generator_output_electricity,...,ID_node2_final,lon2_final,lat2_final,osmID_node1,osmID_node2,lon1_final_1,lat1_final_1,lon2_final_1,lat2_final_1,LineID
0,153608701,line,110000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,1.0,609979.512105,2.317122e+06,153608703,1.0,610733.414911,2.315124e+06,609979.512105,2.317122e+06,LINEVN0001a
1,153608701,line,110000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,1.0,609979.512105,2.317122e+06,153608703,1.0,610733.414911,2.315124e+06,609979.512105,2.317122e+06,LINEVN0001b
2,176708144,line,110000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,3.0,651889.179899,1.285444e+06,2.0,1197858380,672622.408581,1.278800e+06,651900.198685,1.285466e+06,LINEVN0002
3,241194661,line,220000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,5.0,684741.069151,2.304331e+06,4.0,241065371,682154.017051,2.315888e+06,684823.251473,2.304280e+06,LINEVN0003a
4,241194661,line,220000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,5.0,684741.069151,2.304331e+06,4.0,241065371,682154.017051,2.315888e+06,684823.251473,2.304280e+06,LINEVN0003b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7884,1321731650,line,220000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,6593.0,780561.152853,1.629737e+06,8654.0,6593.0,731196.555863,1.654755e+06,780561.152853,1.629737e+06,LINEVN5690b
7885,1321731651,line,220000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,6593.0,780561.152853,1.629737e+06,1321731649,6593.0,781004.779339,1.630088e+06,780561.152853,1.629737e+06,LINEVN5691a
7886,1321731651,line,220000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,6593.0,780561.152853,1.629737e+06,1321731649,6593.0,781004.779339,1.630088e+06,780561.152853,1.629737e+06,LINEVN5691b
7887,1321731651,line,220000,NaN,NaN,50,NaN,NaN,NaN,NaN,...,6593.0,780561.152853,1.629737e+06,1321731649,6593.0,781004.779339,1.630088e+06,780561.152853,1.629737e+06,LINEVN5691c


In [30]:
# node_ids = pd.concat([data_cloned_way['osmID_node1'], data_cloned_way['osmID_node2']]).unique()
# lon_values = pd.concat([data_cloned_way['lon1_final_1'], data_cloned_way['lon2_final_1']]).unique()
# lat_values = pd.concat([data_cloned_way['lat1_final_1'], data_cloned_way['lat2_final_1']]).unique()


In [31]:
export_to_excel(data_cloned_way)

Start exporting data to Excel files... (may take a few seconds)
INFO: Exported lines to ../outputs/tbl_Lines_VN.xlsx
INFO: Exported Nodes to ../outputs/tbl_Nodes_VN.xlsx
... finished!


In [32]:
# def save_to_excel(data, export_excel_country_code='VN'):
#     """
#     将数据保存为 Excel 文件，包含 table_lines 和 table_nodes 两个表格。

#     Parameters:
#     - data: DataFrame 包含原始数据
#     - export_excel_country_code: 国家代码，用于文件命名

#     Returns:
#     - None
#     """

#     data['fromNode'] = data['osmID_node1']
#     data['toNode'] = data['osmID_node2']

#     # 为 Annotation 列添加注释信息
#     data['Annotation'] = ''
#     for index, row in data.iterrows():
#         annotations = []
#         if row['vlevels'] != 1:
#             annotations.append("multiple vlevels")
#         if row['systems'] == 2:
#             annotations.append("6 cables - 2 systems")
#         elif row['systems'] == 3:
#             annotations.append("9 cables - 3 systems")
#         elif row['systems'] == 4:
#             annotations.append("12 cables - 4 systems")
#         if row['dc_candidate']:
#             annotations.append("potentially DC")
#         data.at[index, 'Annotation'] = ', '.join(annotations) if annotations else ' '

#     # 数据列的准备和顺序
#     data['Voltage'] = data['voltage'] / 1000  # 转换为 kV
#     data['Country'] = export_excel_country_code
#     data['R'] = ''
#     data['XL'] = ''
#     data['XC'] = ''
#     data['Itherm'] = ''
#     data['Capacity'] = ''

#     table_lines = data.drop(columns=['voltage'])
#     desired_order = ['Country', 'osm_id', 'LineID', 'fromNode', 'toNode', 'Voltage', 'Length', 
#                      'R', 'XL', 'XC', 'Itherm', 'Capacity', 
#                      'frequency', 'Annotation', 'geometry']
#     other_columns = [col for col in table_lines.columns if col not in desired_order]
#     table_lines = table_lines[desired_order + other_columns]

#     # 导出线路数据至 Excel
#     timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M')
#     filename_lines = f"../outputs/tbl_Lines_{export_excel_country_code}.xlsx"
#     table_lines.to_excel(filename_lines, index=False)
#     print(f'INFO: Exported lines to {filename_lines}')

#     # 准备节点数据并导出至 Excel
#     node1_data = data[['osmID_node1', 'Voltage', 'lon1_final_1', 'lat1_final_1']].rename(columns={
#         'osmID_node1': 'NodeID', 'lon1_final_1': 'lon', 'lat1_final_1': 'lat'})
#     node2_data = data[['osmID_node2', 'Voltage', 'lon2_final_1', 'lat2_final_1']].rename(columns={
#         'osmID_node2': 'NodeID', 'lon2_final_1': 'lon', 'lat2_final_1': 'lat'})
#     table_nodes = pd.concat([node1_data, node2_data]).drop_duplicates(subset='NodeID').reset_index(drop=True)
#     table_nodes['geometry'] = table_nodes.apply(lambda row: Point(row['lon'], row['lat']) if pd.notnull(row['lon']) and pd.notnull(row['lat']) else None, axis=1)

#     filename_nodes = f"../outputs/tbl_Nodes_{export_excel_country_code}.xlsx"
#     table_nodes.to_excel(filename_nodes, index=False)
#     print(f'INFO: Exported Nodes to {filename_nodes}')


# def save_to_gpkg(data):
#     """
#     将数据保存为 GeoPackage 文件，包括线路和节点数据。

#     Parameters:
#     - data: DataFrame 包含原始数据

#     Returns:
#     - None
#     """
#     gdf_lines = gpd.GeoDataFrame(data, geometry='geometry', crs='EPSG:32648')

#     # 准备节点数据
#     node1_data = data[['osmID_node1', 'Voltage', 'lon1_final_1', 'lat1_final_1']].rename(columns={
#         'osmID_node1': 'NodeID', 'lon1_final_1': 'lon', 'lat1_final_1': 'lat'})
#     node2_data = data[['osmID_node2', 'Voltage', 'lon2_final_1', 'lat2_final_1']].rename(columns={
#         'osmID_node2': 'NodeID', 'lon2_final_1': 'lon', 'lat2_final_1': 'lat'})
#     table_nodes = pd.concat([node1_data, node2_data]).drop_duplicates(subset='NodeID').reset_index(drop=True)
#     table_nodes['geometry'] = table_nodes.apply(lambda row: Point(row['lon'], row['lat']) if pd.notnull(row['lon']) and pd.notnull(row['lat']) else None, axis=1)
#     gdf_nodes = gpd.GeoDataFrame(table_nodes, geometry='geometry', crs='EPSG:32648')

#     # 保存为 GeoPackage 文件
#     gdf_lines.to_file("../outputs/table_lines.gpkg", layer='lines', driver='GPKG')
#     gdf_nodes.to_file("../outputs/table_nodes.gpkg", layer='nodes', driver='GPKG')
#     print('INFO: Exported to GeoPackage files: table_lines.gpkg and table_nodes.gpkg')


